In [35]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools
import random
import requests
import os
import json
import time
import psutil

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [36]:
import requests

index_url = 'https://api.github.com/repos/numenta/NAB/contents/data'

# Fetching file names from the index URL
response = requests.get(index_url)

# Check if the response was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    index_data = response.json()
    
    # Extract directory names
    directories = [file['name'] for file in index_data if file.get('type') == "dir"]
    print(directories)
else:
    print("Failed to fetch data:", response.status_code)

['artificialNoAnomaly', 'artificialWithAnomaly', 'realAWSCloudwatch', 'realAdExchange', 'realKnownCause', 'realTraffic', 'realTweets']


In [37]:
base_url = 'https://raw.githubusercontent.com/numenta/NAB/master/data/'
data = {}

def addFolderAndReadAll(d_name):
    data[d_name] = {}
    response = requests.get(index_url + '/' + d_name)
    index_data = response.json()

    csv_files = [ file['name'] for file in index_data if file['type'] == "file"]
    csvs_num = 0
    for f_name in csv_files:
        data[d_name][f_name] = pd.read_csv(base_url + d_name + '/' + f_name)
        csvs_num += 1
    return csvs_num

csvs_num = sum([addFolderAndReadAll(d_name) for d_name in directories])

In [38]:
# Function to get a random start date from the DataFrame index
def get_random_start_date(index):
    return np.random.choice(index)

# Main function to repeat the process until non-None frequency is obtained
def find_non_none_frequency(df, offset=9):
    while True:
        # Get a random start date from the DataFrame index
        start_date = pd.to_datetime(get_random_start_date(df.index))

        # Find the index of the end date by moving 9 steps through the indices
        end_date_index = df.index.get_loc(start_date) + offset

        # Check if the end date index is within the range of the DataFrame index
        if end_date_index < len(df.index):
            # Calculate the end date using the index
            end_date = df.index[end_date_index]

            # Infer frequency within the specified date range
            subset_df = df.loc[start_date:end_date]
            freq = pd.infer_freq(subset_df.index)

            if freq is not None:
                print("Inferred frequency within range", start_date, "-", end_date, ":", freq)
                return freq  # Exit the loop and return the inferred frequency

In [39]:
def max_consecutive_missing_dates(inferred_freq, missing_dates):
    # Function to check if two dates are consecutive based on the inferred frequency
    def are_consecutive(date1, date2, freq):
        # Calculate the difference between dates based on the inferred frequency
        diff = date2 - date1
        # Check if the difference matches the frequency
        if freq == 'D':
            return diff.days == 1
        elif freq.endswith('H')| freq.endswith('h'):
             # If the frequency ends with 'H', check if it represents hourly intervals
            if freq[:-1]:  # Check if there is a multiplier
                  interval = int(freq[:-1])
                  return diff.total_seconds() == interval * 3600
            else:
                   # If no multiplier is provided, it's assumed to be one hour
                   return diff.total_seconds() == 3600
        elif freq.endswith('T') | freq.endswith('min') :
            if freq.endswith('T'):
                # Extract the interval from the frequency string
                interval = int(freq[:-1])
                return diff.seconds // 60 == interval
            else:
                interval = int(freq[:-3])
                return diff.seconds // 60 == interval
        else:
            raise ValueError("Unsupported frequency: {}".format(freq))

    # Initialize variables to track maximum length and current length
    max_consecutive_missing = 0
    current_consecutive_missing = 0

    # Iterate over the missing dates
    for i in range(1, len(missing_dates)):
        # Check if the current date is consecutive with the previous date
        if are_consecutive(missing_dates[i - 1], missing_dates[i], inferred_freq):
            # Increment current consecutive missing count
            current_consecutive_missing += 1
        else:
            # Update maximum consecutive missing count if needed
            max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)
            # Reset current consecutive missing count
            current_consecutive_missing = 0

    # Update max_consecutive_missing if current_consecutive_missing is still greater
    max_consecutive_missing = max(max_consecutive_missing, current_consecutive_missing)

    return max_consecutive_missing

In [40]:
def preprocess(df, f_name):
    # Convert 'timestamp' column to datetime format and rename it to 'ds'
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Removing the duplicate rows
    df = df[~df.duplicated(keep='first')]

    duplicated_dates_length = len(df[df['timestamp'].duplicated(keep=False)])

    if  duplicated_dates_length > 0:
      print("Number of Duplicated Dates in "+ f_name + ": "+ str(duplicated_dates_length))
      # To make the mean as the value for the numerical columns if there are different values for a particular date
      df = df.groupby('timestamp').mean()
      # Reset index to bring 'timestamp' column back
      df.reset_index(inplace=True)

    df.set_index(['timestamp'], inplace=True)
    df.sort_index()

    # Create a date range with hourly frequency covering the entire time range
    start_date = df.index.min()
    end_date = df.index.max()

    #inferred_freq = pd.infer_freq(df.index)
    inferred_freq = find_non_none_frequency(df)

    if inferred_freq is None:
      inferred_freq = default_freq # setting the default frequency
      print("Cannot infer the frequency of the timestamp of the dataset "+ f_name+ " .Therefore the default frequency of " + default_freq+ " will be used")

    expected_date_range = pd.date_range(start=start_date, end=end_date, freq=inferred_freq)

    # Find the missing date entries
    missing_dates = expected_date_range[~expected_date_range.isin(df.index)]
    # Print or work with the list of missing dates
    print("Number of Missing Dates in "+ f_name + ": "+ str(len(missing_dates))+"\n")

    if len(missing_dates) > 0:
      df = df.asfreq(inferred_freq)
      df.sort_index()

      # Call the function with inferred_freq and missing_dates parameters
      max_consecutive = max_consecutive_missing_dates(inferred_freq, missing_dates)
      print("Maximum length of consecutive missing dates:", max_consecutive)
      if max_consecutive > 3:
        print("It is better to use other imputation method rather than linear interpolation")

      df['value'] = df['value'].interpolate(method='linear')

    return df

In [41]:
url = 'https://raw.githubusercontent.com/numenta/NAB/master/labels/combined_labels.json'

response = requests.get(url)

if response.status_code == 200:
    labels = json.loads(response.text)
else:
    print("Failed to retrieve data from the URL:", response.status_code)

In [42]:
# List of directories
dirs = ['realAdExchange', 'realAWSCloudwatch', 'realKnownCause', 'realTweets', 'artificialWithAnomaly', 'artificialNoAnomaly']
#dirs = ['realAdExchange']
#dirs = ['artificialNoAnomaly']

# Loop through each directory
for dir in dirs:
    for f_name in data[dir]:
        print("")
        print(f"Iterating over file: {dir} / {f_name}")
        df = preprocess(data[dir][f_name], f_name)
        labels_of_one_file = labels[dir+'/'+f_name]
        df['is_anomaly'] = 0
        for anomalous_timestamp in labels_of_one_file:
            anomalous_timestamp = pd.to_datetime(anomalous_timestamp)
            try:
                df.at[anomalous_timestamp, 'is_anomaly'] = 1  # Set is_anomaly to 1 at the index location
            except KeyError:
                print(f"Anomalous timestamp {anomalous_timestamp} not found in data[{dir}][{f_name}].")
                pass
        data[dir][f_name] = df  # Assign the modified DataFrame back to the data dictionary


Iterating over file: realAdExchange / exchange-2_cpc_results.csv
Number of Duplicated Dates in exchange-2_cpc_results.csv: 2
Inferred frequency within range 2011-08-19 09:00:01 - 2011-08-19 18:00:01 : h
Number of Missing Dates in exchange-2_cpc_results.csv: 25

Maximum length of consecutive missing dates: 19
It is better to use other imputation method rather than linear interpolation

Iterating over file: realAdExchange / exchange-2_cpm_results.csv
Number of Duplicated Dates in exchange-2_cpm_results.csv: 2
Inferred frequency within range 2011-07-04 23:00:01 - 2011-07-05 08:00:01 : h
Number of Missing Dates in exchange-2_cpm_results.csv: 25

Maximum length of consecutive missing dates: 19
It is better to use other imputation method rather than linear interpolation

Iterating over file: realAdExchange / exchange-3_cpc_results.csv
Inferred frequency within range 2011-08-05 06:15:01 - 2011-08-05 15:15:01 : h
Number of Missing Dates in exchange-3_cpc_results.csv: 109

Maximum length of co

In [43]:
# Specify the directory and file name
dir = 'realAWSCloudwatch'
f_name = 'ec2_cpu_utilization_ac20cd.csv'


# Retrieve the DataFrame
df = data[dir][f_name]

In [44]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-04-15')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df


In [45]:
f_name = 'ec2_cpu_utilization_5f5533.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [46]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [47]:
f_name = 'grok_asg_anomaly.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [48]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-01-29')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [49]:
f_name = 'rds_cpu_utilization_cc0c53.csv'

# Retrieve the DataFrame
df = data[dir][f_name]

In [50]:
# Define the date to crop the DataFrame
cutoff_date = pd.Timestamp('2014-02-25')

# Filter the DataFrame using the index and store it back to the same variable
df = df[df.index <= cutoff_date]

# Store the filtered DataFrame back into the dictionary
data[dir][f_name] = df

In [51]:
def split_data(df, train_ratio=0.6):
    train_size = int(len(df) * train_ratio)
    train, val = df[:train_size], df[train_size:]
    return train, val

In [52]:
# Create an empty DataFrame with the desired columns
original_values_df = pd.DataFrame(columns=['dir', 'values'])

for dir in dirs:
    print(f"Iterating over directory: {dir}")

    # Iterate over each file in the current directory
    for file_name in data[dir]:
        df = data[dir][file_name]
        # Assuming df is defined somewhere in the loop
        train, val = split_data(df)

        # Create a new row with the directory and values as a list
        new_row = pd.DataFrame({'dir': [dir], 'file_name':[file_name], 'values': [val['value'].tolist()]})

        # Concatenate the new row to the original_values_df
        original_values_df = pd.concat([original_values_df, new_row], ignore_index=True)


original_values_df

Iterating over directory: realAdExchange
Iterating over directory: realAWSCloudwatch
Iterating over directory: realKnownCause
Iterating over directory: realTweets
Iterating over directory: artificialWithAnomaly
Iterating over directory: artificialNoAnomaly


,dir,values,file_name
0,realAdExchange,"[0.0848198198198, 0.0985912823752, 0.111722365...",exchange-2_cpc_results.csv
1,realAdExchange,"[0.160230773158, 0.210830475115, 0.28622422434...",exchange-2_cpm_results.csv
2,realAdExchange,"[0.0970408163265, 0.0936600833484, 0.094246275...",exchange-3_cpc_results.csv
3,realAdExchange,"[0.56535245238, 0.545612490685, 0.521651179882...",exchange-3_cpm_results.csv
4,realAdExchange,"[0.0737386325362, 0.0840131819846, 0.079394993...",exchange-4_cpc_results.csv
5,realAdExchange,"[0.428944811047, 0.469532759242, 0.45564022899...",exchange-4_cpm_results.csv
6,realAWSCloudwatch,"[0.132, 0.066, 0.2, 0.132, 0.134, 0.132, 0.066...",ec2_cpu_utilization_24ae8d.csv
7,realAWSCloudwatch,"[1.834, 2.042, 1.8, 1.826, 1.764, 1.7380000000...",ec2_cpu_utilization_53ea38.csv
8,realAWSCloudwatch,"[47.018, 43.004, 41.066, 44.554, 43.284, 42.12...",ec2_cpu_utilization_5f5533.csv
9,realAWSCloudwatch,"[0.102, 0.102, 81.468, 98.67, 60.688, 87.11200...",ec2_cpu_utilization_77c1ca.csv


Loading of datasets (time series extracted features)

In [53]:
df = pd.read_csv('../dataset_preparation/ranking_dataset.csv')
df.head()

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,arima,xgboost,random_forest
0,artificialNoAnomaly/art_daily_no_noise.csv,NaN,4032,-45.198782,0.046765,5.000000e-01,1.000000e-04,2.500000e-01,1.050000e-62,1,...,8.854128,-5.440000e-14,2.970000e-26,-0.500000,0.250000,0.928571,22.402977,25.282784,0.011355,0.003701
1,artificialNoAnomaly/art_daily_perfect_square_w...,NaN,4032,-63.887428,0.043267,5.000000e-01,1.750000e-01,2.000000e-01,0.000000e+00,1,...,8.440207,0.000000e+00,0.000000e+00,-0.500000,0.250000,0.928571,22.385788,26.668146,0.009475,0.000024
2,artificialNoAnomaly/art_daily_small_noise.csv,0.412716,4032,-57.839587,0.046790,1.490000e-08,1.010000e-08,3.110000e-21,1.280000e-05,1,...,8.643415,-2.737291e-01,8.206679e-02,-0.620340,0.418001,0.918527,22.720717,25.473923,2.009846,2.076357
3,artificialNoAnomaly/art_flatline.csv,NaN,4032,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,45.000000,0.000000,0.000000
4,artificialNoAnomaly/art_noisy.csv,0.507001,4032,-4126.670693,0.299753,1.580000e-08,7.610000e-09,2.480000e-06,5.353556e-03,1,...,0.001607,-5.131762e-01,2.664287e-01,-0.676908,0.497557,0.005106,2.872416,2.820160,2.820342,2.883200


In [54]:
predicted_df_final = pd.read_csv('../dataset_preparation/predicted_results.csv')
predicted_df_final.head()

,dir,file_name,original_value,exponential_smoothing,xgboost,random_forest
0,realAdExchange,exchange-2_cpc_results.csv,[0.08481982 0.09859128 0.11172237 0.14488593 0...,"[0.08710862616837661, 0.09597503133097246, 0.1...","[0.0836484283208847, 0.09625387191772461, 0.09...","[0.08690806478261948, 0.10730652511119843, 0.1..."
1,realAdExchange,exchange-2_cpm_results.csv,[1.60230773e-01 2.10830475e-01 2.86224224e-01 ...,"[0.2142570156362756, 0.26667298300939435, 0.23...","[0.16946975886821747, 0.2190512865781784, 0.22...","[0.14904223382472992, 0.2034716159105301, 0.24..."
2,realAdExchange,exchange-3_cpc_results.csv,[0.09704082 0.09366008 0.09424628 0.11277086 0...,"[0.09775771566764195, 0.08847114527361032, 0.0...","[0.08921685814857483, 0.08921685814857483, 0.0...","[0.09203566610813141, 0.09147148579359055, 0.0..."
3,realAdExchange,exchange-3_cpm_results.csv,[0.56535245 0.54561249 0.52165118 0.57706644 0...,"[0.6415997673327005, 0.5748525083557499, 0.547...","[0.5548130869865417, 0.5548130869865417, 0.554...","[0.541362464427948, 0.5110874772071838, 0.5241..."
4,realAdExchange,exchange-4_cpc_results.csv,[0.07373863 0.08401318 0.07939499 0.07175556 0...,"[0.08247547533245321, 0.08762157084060164, 0.0...","[0.07643906772136688, 0.07643906772136688, 0.0...","[0.07837941497564316, 0.07795900851488113, 0.0..."


In [55]:
#original_values_df = pd.read_csv('../dataset_preparation/original_values.csv')
#original_values_df.head()

In [56]:
mae_df_final = pd.read_csv('../dataset_preparation/mae_results.csv')
mae_df_final.head()

,dir,file_name,exponential_smoothing,xgboost,random_forest
0,realAdExchange,exchange-2_cpc_results.csv,0.015797,0.017877,0.021553
1,realAdExchange,exchange-2_cpm_results.csv,0.066420,0.068205,0.126764
2,realAdExchange,exchange-3_cpc_results.csv,0.035462,0.043285,0.040023
3,realAdExchange,exchange-3_cpm_results.csv,0.194866,0.280655,0.242409
4,realAdExchange,exchange-4_cpc_results.csv,0.041684,0.039525,0.043989


In [57]:
mape_df_final = pd.read_csv('../dataset_preparation/mape_results.csv')
mape_df_final.head()

,dir,file_name,exponential_smoothing,xgboost,random_forest
0,realAdExchange,exchange-2_cpc_results.csv,0.145130,0.161013,0.200096
1,realAdExchange,exchange-2_cpm_results.csv,0.892094,0.839719,0.934698
2,realAdExchange,exchange-3_cpc_results.csv,0.207747,0.259299,0.225980
3,realAdExchange,exchange-3_cpm_results.csv,0.179405,0.274153,0.232651
4,realAdExchange,exchange-4_cpc_results.csv,0.411442,0.316447,0.391400


Ranking of base models based on the evaluation metrics

In [58]:
# Define a function to rank models based on MAE values for each row
def rank_models(row):
    mae_values = row[['exponential_smoothing','xgboost', 'random_forest']]
    #mae_values = row[['exponential_smoothing', 'xgboost', 'random_forest']]
    model_rank = mae_values.sort_values().index.tolist()
    return model_rank

In [59]:
# Apply the function to each row of the DataFrame
df['model_rank'] = df.apply(rank_models, axis=1)
df.head()

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,arima,xgboost,random_forest,model_rank
0,artificialNoAnomaly/art_daily_no_noise.csv,NaN,4032,-45.198782,0.046765,5.000000e-01,1.000000e-04,2.500000e-01,1.050000e-62,1,...,-5.440000e-14,2.970000e-26,-0.500000,0.250000,0.928571,22.402977,25.282784,0.011355,0.003701,"[random_forest, xgboost, exponential_smoothing]"
1,artificialNoAnomaly/art_daily_perfect_square_w...,NaN,4032,-63.887428,0.043267,5.000000e-01,1.750000e-01,2.000000e-01,0.000000e+00,1,...,0.000000e+00,0.000000e+00,-0.500000,0.250000,0.928571,22.385788,26.668146,0.009475,0.000024,"[random_forest, xgboost, exponential_smoothing]"
2,artificialNoAnomaly/art_daily_small_noise.csv,0.412716,4032,-57.839587,0.046790,1.490000e-08,1.010000e-08,3.110000e-21,1.280000e-05,1,...,-2.737291e-01,8.206679e-02,-0.620340,0.418001,0.918527,22.720717,25.473923,2.009846,2.076357,"[xgboost, random_forest, exponential_smoothing]"
3,artificialNoAnomaly/art_flatline.csv,NaN,4032,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,0.000000,45.000000,0.000000,0.000000,"[exponential_smoothing, xgboost, random_forest]"
4,artificialNoAnomaly/art_noisy.csv,0.507001,4032,-4126.670693,0.299753,1.580000e-08,7.610000e-09,2.480000e-06,5.353556e-03,1,...,-5.131762e-01,2.664287e-01,-0.676908,0.497557,0.005106,2.872416,2.820160,2.820342,2.883200,"[xgboost, exponential_smoothing, random_forest]"


Stacking Appraoch to combine inputs from base model - RandomForest Regressor 

In [60]:
def stacked_model_predictions(val, base_preds):
     # Convert string inputs to lists of floats
     # Convert string inputs to lists of floats for each column in base_preds
    base_preds = [list(map(float, col.strip('[]').split(','))) for col in base_preds]
    
    # Convert lists to numpy arrays and transpose to get the correct shape
    base_preds = np.asarray(base_preds).T

    print("Type of base_preds:", type(base_preds))
    print("Shape of base_preds:", base_preds.shape)
    #val = list(map(float, val[0].strip('[]').split(',')))
    
    
    
    # Convert lists to numpy arrays
    val = np.asarray(val).reshape(-1)
    #base_preds = np.asarray(base_preds).reshape(-1, 1)
    # Check the type and shape after conversion
    print("Type of val:", type(val))
    print("Shape of val:", val.shape)
    

    # Check if base_preds and val have the same length
    if base_preds.shape[0] != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Ensure base_preds and val are numpy arrays

    
    # Check if base_preds and val have the same length
    if len(base_preds) != len(val):
        raise ValueError("base_preds and val must have the same length.")
    # Splitting features and target variable sequentially
    train_size = int(0.65 * len(val))  # Assuming the split ratio is 80-20
    X_train, y_train = base_preds[:train_size], val[:train_size]
    X_val, y_val = base_preds[train_size:], val[train_size:]

    # Define parameter grid for Random Forest
    param_grid = {
        'n_estimators': [25, 50, 100, 150, 200],  # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],      # Maximum depth of the tree
        'min_samples_split': [2, 5, 8, 10, 15],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4, 6]     # Minimum number of samples required to be at a leaf node
    }

    # Initialize Random Forest regressor
    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=2, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    print("Stacking Approach")

    # Print the best estimator found
    print(search.best_estimator_)

    # Make predictions using the best model
    y_pred = search.best_estimator_.predict(X_val)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_val, y_pred)
    print("Mean Absolute Error (MAE):", mae)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error (MSE):", mse)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error (MAPE):", mape)

    print("")

    return y_pred, y_val, mae, mse, rmse, mape


In [61]:
# Create an empty column named 'ensemble_size'
df['ensemble_size'] = np.nan
# Display the DataFrame with the new empty column
df.head(2)

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,arima,xgboost,random_forest,model_rank,ensemble_size
0,artificialNoAnomaly/art_daily_no_noise.csv,NaN,4032,-45.198782,0.046765,0.5,0.0001,0.25,1.050000e-62,1,...,2.970000e-26,-0.5,0.25,0.928571,22.402977,25.282784,0.011355,0.003701,"[random_forest, xgboost, exponential_smoothing]",NaN
1,artificialNoAnomaly/art_daily_perfect_square_w...,NaN,4032,-63.887428,0.043267,0.5,0.1750,0.20,0.000000e+00,1,...,0.000000e+00,-0.5,0.25,0.928571,22.385788,26.668146,0.009475,0.000024,"[random_forest, xgboost, exponential_smoothing]",NaN


Logic to find the appropriate ensemble size for training

In [62]:
def find_ensemble_size(model_rank, unique_id, predicted_df_final, mae_df_final, original_values_df):

    threshold_mae=0.05
    # Extract directory and file name from unique_id
    dir_name, file_name = unique_id.split('/')

    # Get the first element from the list of model_rank
    model_name = model_rank[0]

    # Find the row in mae_df dataframe that matches the directory and file name
    row = mae_df_final[(mae_df_final['dir'] == dir_name) & (mae_df_final['file_name'] == file_name)]

    # Find the value in the column that matches the model_name
    model_mae = row[model_name].iloc[0]

    # Determine the ensemble size based on the model MAE value
    if model_mae < threshold_mae:
        print(f"No need for stacking approach, since the first model {model_name} has MAE value {model_mae} less than the threshold value {threshold_mae}")
        return 1
    else:
        previous_mae = model_mae
        print("Going for stacking Approach")
        i = 1  # Initialize the count of models
        while i < len(model_rank):
            print("Going for the next iteration ", i+1, " of the same loop")
            threshold_mae=threshold_mae+0.05
            i += 1  # Increment the count of models
            models_to_use = model_rank[:i]  # Take the first i models from the model_rank list
            base_preds = []  # Initialize base_preds as a list
            # Get the predicted values for the selected models
            for model in models_to_use:
                # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model].iloc[0]
                # Append the predicted value to base_preds
                base_preds.append(pred_value)

            val_row = original_values_df[(original_values_df['dir'] == dir_name) & (original_values_df['file_name'] == file_name)]
            val = val_row['values'].iloc[0]

            base_preds = np.stack(base_preds, axis=-1)
            y_pred, y_val, mae, mse, rmse, mape = stacked_model_predictions(val, base_preds)

            if mae > previous_mae:
                print("MAE increased after adding", i, "models, from", previous_mae, "to", mae, "so returning the previous ensemble size")
                return i - 1

            # Update previous MAE with current MAE
            previous_mae = mae

            # If MAE is less than 0.05, return the current ensemble size
            if mae < threshold_mae:
                return i        

    # If none of the models have MAE less than 0.05, return the total count of models
    return len(model_rank)

In [63]:
for index, row in df.iterrows():
    # Extract values from the current row
    model_rank = row['model_rank']
    unique_id = row['unique_id']

    # Print the unique ID before calling the function
    print("Unique ID:", unique_id)

    # Call the function to find ensemble size
    ensemble_size = find_ensemble_size(model_rank, unique_id, predicted_df_final, mae_df_final, original_values_df)

    print("")

    # Assign the ensemble size to the 'ensemble_size' column
    df.at[index, 'ensemble_size'] = ensemble_size

Unique ID: artificialNoAnomaly/art_daily_no_noise.csv
No need for stacking approach, since the first model random_forest has MAE value 0.0037471047465226 less than the threshold value 0.05

Unique ID: artificialNoAnomaly/art_daily_perfect_square_wave.csv
No need for stacking approach, since the first model random_forest has MAE value 2.7802592098380348e-05 less than the threshold value 0.05

Unique ID: artificialNoAnomaly/art_daily_small_noise.csv
Going for stacking Approach
Going for the next iteration  2  of the same loop
Type of base_preds: <class 'numpy.ndarray'>
Shape of base_preds: (1613, 2)
Type of val: <class 'numpy.ndarray'>
Shape of val: (1613,)


Stacking Approach
RandomForestRegressor(max_depth=30, min_samples_leaf=4, min_samples_split=15,
                      n_estimators=200, random_state=42)
Mean Absolute Error (MAE): 2.1751633367938696
Mean Squared Error (MSE): 8.814916392301377
Root Mean Squared Error (RMSE): 2.9689924877475486
Mean Absolute Percentage Error (MAPE): 0.05144260972105995

Going for the next iteration  3  of the same loop
Type of base_preds: <class 'numpy.ndarray'>
Shape of base_preds: (1613, 3)
Type of val: <class 'numpy.ndarray'>
Shape of val: (1613,)
Stacking Approach
RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=15,
                      n_estimators=150, random_state=42)
Mean Absolute Error (MAE): 2.215310998950682
Mean Squared Error (MSE): 9.240690535515789
Root Mean Squared Error (RMSE): 3.0398504133453326
Mean Absolute Percentage Error (MAPE): 0.052396663352360366

MAE increased after adding 3 models, from 2.1751633367938696 to 2.215310998950682 so returning the previous 

In [64]:
# Display the updated DataFrame
df

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,arima,xgboost,random_forest,model_rank,ensemble_size
0,artificialNoAnomaly/art_daily_no_noise.csv,NaN,4032,-45.198782,0.046765,5.000000e-01,1.000000e-04,2.500000e-01,1.050000e-62,1,...,2.970000e-26,-0.500000,0.250000,0.928571,2.240298e+01,2.528278e+01,1.135547e-02,3.701070e-03,"[random_forest, xgboost, exponential_smoothing]",1.0
1,artificialNoAnomaly/art_daily_perfect_square_w...,NaN,4032,-63.887428,0.043267,5.000000e-01,1.750000e-01,2.000000e-01,0.000000e+00,1,...,0.000000e+00,-0.500000,0.250000,0.928571,2.238579e+01,2.666815e+01,9.474617e-03,2.440000e-05,"[random_forest, xgboost, exponential_smoothing]",1.0
2,artificialNoAnomaly/art_daily_small_noise.csv,0.412716,4032,-57.839587,0.046790,1.490000e-08,1.010000e-08,3.110000e-21,1.280000e-05,1,...,8.206679e-02,-0.620340,0.418001,0.918527,2.272072e+01,2.547392e+01,2.009846e+00,2.076357e+00,"[xgboost, random_forest, exponential_smoothing]",2.0
3,artificialNoAnomaly/art_flatline.csv,NaN,4032,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,0.000000e+00,4.500000e+01,0.000000e+00,0.000000e+00,"[exponential_smoothing, xgboost, random_forest]",1.0
4,artificialNoAnomaly/art_noisy.csv,0.507001,4032,-4126.670693,0.299753,1.580000e-08,7.610000e-09,2.480000e-06,5.353556e-03,1,...,2.664287e-01,-0.676908,0.497557,0.005106,2.872416e+00,2.820160e+00,2.820342e+00,2.883200e+00,"[xgboost, exponential_smoothing, random_forest]",1.0
5,artificialWithAnomaly/art_daily_flatmiddle.csv,0.501646,4032,-49.327420,0.121571,5.372404e-01,1.590000e-07,2.630000e-07,1.456821e-02,1,...,1.705510e-02,-0.553618,0.314774,0.833093,3.344287e+01,3.906735e+01,1.302283e+01,1.304678e+01,"[xgboost, random_forest, exponential_smoothing]",1.0
6,artificialWithAnomaly/art_daily_jumpsdown.csv,0.523080,4032,-55.541723,0.098380,2.378961e-01,2.540000e-13,1.590000e-12,2.104731e-02,1,...,7.690519e-02,-0.611336,0.393784,0.861524,2.135077e+01,2.237935e+01,5.233680e+00,5.305939e+00,"[xgboost, random_forest, exponential_smoothing]",2.0
7,artificialWithAnomaly/art_daily_jumpsup.csv,0.565294,4032,-53.046075,0.260825,3.412410e-01,8.780000e-11,7.510000e-08,4.711886e-02,1,...,7.479507e-02,-0.619568,0.409203,0.817634,2.898232e+01,3.144872e+01,7.966740e+00,7.984297e+00,"[xgboost, random_forest, exponential_smoothing]",1.0
8,artificialWithAnomaly/art_daily_nojump.csv,0.553390,4032,-52.591420,0.185286,3.270213e-01,1.100000e-10,7.620000e-08,4.757963e-02,1,...,5.806725e-02,-0.599234,0.371903,0.796279,2.230242e+01,2.372645e+01,7.032782e+00,7.075534e+00,"[xgboost, random_forest, exponential_smoothing]",2.0
9,artificialWithAnomaly/art_increase_spike_densi...,0.324410,4032,-4751.528222,0.008390,1.500000e-08,1.040000e-08,5.360000e-09,4.910010e-04,1,...,5.635558e-01,-0.661972,0.877211,-0.020033,8.970895e-01,8.120496e-01,9.537371e-01,1.258561e+00,"[exponential_smoothing, xgboost, random_forest]",2.0


In [65]:
# Assuming df_features is your DataFrame

# Convert 'ensemble_size' column from float to integer
df['ensemble_size'] = df['ensemble_size'].astype(int)

# Print the DataFrame to verify the changes
df.head(5)

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,exponential_smoothing,arima,xgboost,random_forest,model_rank,ensemble_size
0,artificialNoAnomaly/art_daily_no_noise.csv,NaN,4032,-45.198782,0.046765,5.000000e-01,1.000000e-04,2.500000e-01,1.050000e-62,1,...,2.970000e-26,-0.500000,0.250000,0.928571,22.402977,25.282784,0.011355,0.003701,"[random_forest, xgboost, exponential_smoothing]",1
1,artificialNoAnomaly/art_daily_perfect_square_w...,NaN,4032,-63.887428,0.043267,5.000000e-01,1.750000e-01,2.000000e-01,0.000000e+00,1,...,0.000000e+00,-0.500000,0.250000,0.928571,22.385788,26.668146,0.009475,0.000024,"[random_forest, xgboost, exponential_smoothing]",1
2,artificialNoAnomaly/art_daily_small_noise.csv,0.412716,4032,-57.839587,0.046790,1.490000e-08,1.010000e-08,3.110000e-21,1.280000e-05,1,...,8.206679e-02,-0.620340,0.418001,0.918527,22.720717,25.473923,2.009846,2.076357,"[xgboost, random_forest, exponential_smoothing]",2
3,artificialNoAnomaly/art_flatline.csv,NaN,4032,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,0.000000,45.000000,0.000000,0.000000,"[exponential_smoothing, xgboost, random_forest]",1
4,artificialNoAnomaly/art_noisy.csv,0.507001,4032,-4126.670693,0.299753,1.580000e-08,7.610000e-09,2.480000e-06,5.353556e-03,1,...,2.664287e-01,-0.676908,0.497557,0.005106,2.872416,2.820160,2.820342,2.883200,"[xgboost, exponential_smoothing, random_forest]",1


In [66]:
# Delete the row with index 3
df = df.drop(index=3)

In [67]:


# Save the DataFrame
df.to_csv('../dataset_preparation/df_features_with_ensemble_size_2.csv', index=False)  # for CSV file

print("CSV saved successfully")

CSV saved successfully
